In [341]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go

py.init_notebook_mode(connected=True)

In [342]:
df = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\Python Scripts\Garmin_Step_Count\Data\May212019-Jan052020\WELLNESS_TOTAL_STEPS.csv',skiprows=[0])

In [343]:
count = 31
for x in range(1, count+1):
    file_num = (x)
    temp = pd.read_csv(fr'C:\Users\samue\OneDrive\Documents\Python Scripts\Garmin_Step_Count\Data\May212019-Jan052020\WELLNESS_TOTAL_STEPS ({file_num}).csv',skiprows=[0])
    df = pd.concat([temp, df], ignore_index=True)

In [344]:
rng = pd.date_range(end='2020-01-05', periods=212, freq='D').sort_values(ascending=True)

df.insert(0,'date',rng)

df.to_csv(r'C:\Users\samue\OneDrive\Documents\Python Scripts\Garmin_Step_Count\Data\Cleaned_data\clean.csv')

In [345]:
def consecutive(count,col):
    """Returns a series of ints that indicates if there have been consecutive negative
    or positive numbers in a row before and including the current item, 1 for yes and 0 for no.
    
    Parameters
    ----------
    count : int
        The number of consecutive to produce a true
    col : pandas series
        The column for comparison
    """
    
    bool_col = col.apply(lambda x: 1 if x >= 0 else 0)
    result = bool_col
    
    for x in range(1, count):
        result = result + bool_col[x:].reset_index(drop=True)
    
    result = result[:-count+1]
    result.index += count-1
    
    result = result.apply(lambda x: 1 if x == count or x == 0 else 0)
    
    return(result)

In [359]:
df['Difference'] = df['Actual'] - df['Goal']
df['Goal_Delta'] = df['Goal'][1:].reset_index(drop=True) - df['Goal']

df['Goal_Diff_Perc'] = df['Actual'] / df['Goal'] * 100

df['Diff_Scale'] = df['Difference'] / df['Goal']
df['Diff_Scale_Norm'] = (df['Diff_Scale']-df['Diff_Scale'].min()) / (df['Diff_Scale'].max()-df['Diff_Scale'].min())

for x in range(2,5):
    df[f'Consec {x}'] = consecutive(x,df['Goal_Delta'])

In [360]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 12 columns):
date               212 non-null datetime64[ns]
Unnamed: 0         212 non-null object
Actual             212 non-null int64
Goal               212 non-null int64
Difference         212 non-null int64
Goal_Delta         211 non-null float64
Goal_Diff_Perc     212 non-null float64
Consec 2           211 non-null float64
Consec 3           210 non-null float64
Consec 4           209 non-null float64
Diff_Scale         212 non-null float64
Diff_Scale_Norm    212 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(3), object(1)
memory usage: 20.0+ KB


In [348]:
df[["Actual","Goal","Difference","Goal_Delta","Goal_Diff_Perc"]] = df[["Actual","Goal","Difference","Goal_Delta","Goal_Diff_Perc"]].apply(pd.to_numeric)

In [362]:
# plt.plot(df['Difference']);
# fig = go.Figure(
#     data=[go.Scatter(y=df['Goal_Delta'], x=df['Goal_Diff_Perc'])]
# )
# fig.show()

fig = px.scatter(df,x='Goal_Delta', y='Difference', color="Consec 2",size='Diff_Scale_Norm')
fig.show()

In [350]:
fig = px.scatter(df,x='Difference', y='Goal_Delta', size="Goal_Diff_Perc", color="Consec 3")
fig.show()

In [361]:
df['Diff_Scale_Norm']

0      0.553097
1      0.338558
2      0.196791
3      0.332770
4      0.420303
5      0.153238
6      0.194355
7      0.052126
8      0.673152
9      0.252902
10     0.413394
11     0.396041
12     0.181682
13     0.331755
14     0.920160
15     0.417816
16     0.357313
17     0.716638
18     0.334304
19     0.244160
20     0.486423
21     0.422551
22     0.702641
23     0.168440
24     0.563528
25     0.235511
26     0.238273
27     0.162981
28     0.840447
29     0.391570
         ...   
182    0.903037
183    0.471257
184    0.354644
185    0.459110
186    0.230782
187    0.189219
188    0.266237
189    0.619894
190    0.434721
191    0.010306
192    0.086689
193    0.295507
194    0.261145
195    0.173351
196    1.000000
197    0.589801
198    0.495160
199    0.303822
200    0.359111
201    0.331286
202    0.334222
203    0.049412
204    0.432282
205    0.361546
206    0.113436
207    0.055416
208    0.437275
209    0.316281
210    0.272033
211    0.000000
Name: Diff_Scale_Norm, L

In [ ]:
#     data=[go.Scatter(y=df['Diffrence'], x=df['Goal'])],
#     layout_title_text="A Figure Displayed with fig.show()"
# )
# fig.show()